<a href="https://colab.research.google.com/github/JordanFoss/STAT3007_Project/blob/main/Conv_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/JordanFoss/STAT3007_Project.git
%cd STAT3007_Project/

Cloning into 'STAT3007_Project'...
remote: Enumerating objects: 3982, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (209/209), done.
remote: Total 3982 (delta 114), reused 168 (delta 49), pack-reused 3722
Receiving objects: 100% (3982/3982), 677.11 MiB | 31.63 MiB/s, done.
Resolving deltas: 100% (699/699), done.
Checking out files: 100% (2855/2855), done.
/content/STAT3007_Project


In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
import scipy
import numpy as np
import librosa
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.autograd import Variable
import pandas as pd
import glob
import os
from librosa import display
from torchsummary import summary

from IPython.display import Audio

In [3]:
!pip install colorednoise as cn

  Created wheel for colorednoise: filename=colorednoise-1.1.1-cp37-none-any.whl size=3958 sha256=f3b42fe4e637d6bef06b8f5fe89102f587fafa13856f25fdecf4b1480591546f
  Stored in directory: /root/.cache/pip/wheels/84/be/f3/3e7e1c80ebab3f6f0dbd3e34e787b902d2280d66706485fef4
Successfully built colorednoise


In [4]:
import pre_process
import data_loading
import CNN_Model
from data_loading import *

In [5]:
X,y = data_loading.load_samples(os.getcwd())

In [6]:
X = torch.tensor(X)
X = X.reshape(X.shape[0],1,X.shape[1],X.shape[2])
y = torch.tensor(y)

In [7]:
X.shape

torch.Size([480, 1, 128, 63])

In [8]:
cnn = CNN_Model.ConvNet()

In [10]:
summary(cnn.cuda(), (1,128,21))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 14, 127, 19]              98
              ReLU-2          [-1, 14, 127, 19]               0
         MaxPool2d-3            [-1, 14, 64, 9]               0
           Dropout-4            [-1, 14, 64, 9]               0
            Conv2d-5            [-1, 24, 63, 7]           2,040
              ReLU-6            [-1, 24, 63, 7]               0
         MaxPool2d-7            [-1, 24, 32, 3]               0
Total params: 2,138
Trainable params: 2,138
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.82
Params size (MB): 0.01
Estimated Total Size (MB): 0.84
----------------------------------------------------------------


In [11]:
# RNN combined with CNN
class LRCN(nn.Module):
    def __init__(self, CNN, shape = (24,32,3)):
        super(LRCN, self).__init__()

        self.cnn = CNN
        self.shape = shape

        channel, freq, times = shape
        self.lstm_layers = nn.LSTM(freq*channel*times,256,num_layers = 2, bidirectional = True)
        self.linear = nn.Sequential(nn.Linear(256*2, 5))
        self.flatten = nn.Flatten()

    def forward(self, x, step_size = 21, use_cuda = False):

      if use_cuda:
        h_t = torch.zeros(4,x.shape[0] ,256, dtype=torch.float).to(x.device)
        c_t = torch.zeros(4,x.shape[0], 256, dtype=torch.float).to(x.device)

      else:
        h_t = torch.zeros(4,x.shape[0], 256, dtype=torch.float)
        c_t = torch.zeros(4,x.shape[0], 256, dtype=torch.float)
      
      look_ahead_time = 21
      for current_time in range(0,x.shape[-1], step_size):

        x_t = x[:,:,:,current_time:current_time+look_ahead_time]
        conv_x = self.cnn(x_t)

        conv_x_flat =  self.flatten(conv_x)

        conv_x_flat = conv_x_flat.reshape(1,conv_x_flat.shape[0],conv_x_flat.shape[1])

        output, (h_t, c_t) = self.lstm_layers(conv_x_flat, (h_t, c_t))

      decision_vec = self.linear(output[0])
      return decision_vec

In [12]:
conv_LSTM = LRCN(cnn)

In [15]:
data_sets = data_loading.load_sets(X,y,train_ratio=[0.8], seed = [10])

In [16]:
data_train, data_test = data_sets[0]

In [17]:
def train_model(data_train, data_test, net, loss, nepoch ,lr = 0.01, batch_size = -1, use_cuda = False, print_output = True):

  # appropriate data type for CPU or GPU
  device = None
  if use_cuda and torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor
    device = torch.device("cuda")
    net = net.to(device)
  else:
    dtype = torch.FloatTensor

  optimizer = optim.SGD(net.parameters(), lr = lr)
  data_train = data_train.change_type(dtype)
  data_test = data_test.change_type(dtype)

  data_loader = DataLoader(data_train, batch_size = batch_size, shuffle = True)

  for epoch in range(nepoch):
    for X_batch, y_batch in data_loader:
      y_batch = y_batch.type(torch.LongTensor)
      if use_cuda and device != None:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_batch = y_batch.type(torch.cuda.LongTensor)

      optimizer.zero_grad()

      # since all our values are negative, we convert them to positive


      pred = net(X_batch, use_cuda = use_cuda)
      Rn = loss(pred, y_batch)
      Rn.backward()
      optimizer.step()

    if print_output:
      print('epoch:', epoch)
      print('loss:',Rn.item())
      print('------------')

  print('final loss:', Rn.item())

  return net

In [18]:
loss = nn.CrossEntropyLoss()

In [19]:
nepoch = 100
trained_net = train_model(data_train, data_test, conv_LSTM, loss, nepoch=nepoch , batch_size = 10,lr = 0.01, use_cuda = True, print_output = True)

epoch: 0
loss: 1.5841327905654907
------------
epoch: 1
loss: 1.59458327293396
------------
epoch: 2
loss: 1.4534306526184082
------------
epoch: 3
loss: 1.494640827178955
------------
epoch: 4
loss: 1.5227930545806885
------------
epoch: 5
loss: 1.3978391885757446
------------
epoch: 6
loss: 1.3122608661651611
------------
epoch: 7
loss: 1.377127766609192
------------
epoch: 8
loss: 1.1024502515792847
------------
epoch: 9
loss: 1.208505392074585
------------
epoch: 10
loss: 1.0481246709823608
------------
epoch: 11
loss: 1.2226552963256836
------------
epoch: 12
loss: 1.284131646156311
------------
epoch: 13
loss: 1.3064134120941162
------------
epoch: 14
loss: 1.2983156442642212
------------
epoch: 15
loss: 0.9761627316474915
------------
epoch: 16
loss: 1.3816237449645996
------------
epoch: 17
loss: 0.8816111087799072
------------
epoch: 18
loss: 1.320134162902832
------------
epoch: 19
loss: 0.9197540879249573
------------
epoch: 20
loss: 0.9899702072143555
------------
epoch: 21

In [24]:
X_test, y_test = data_test.get_data()
trained_net = trained_net.to(torch.device('cpu'))
pred = trained_net(X_test)

In [26]:
accur = CNN_Model.accuracy(pred,y_test)

In [27]:
print(accur)

0.6354166865348816
